<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

# Задание 3

## Классификация текстов

В этом задании вам предстоит попробовать несколько методов, используемых в задаче классификации, а также понять насколько хорошо модель понимает смысл слов и какие слова в примере влияют на результат.

In [1]:
!pip uninstall torchtext
!pip install torchtext==0.8.1

Found existing installation: torchtext 0.13.1
Uninstalling torchtext-0.13.1:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/torchtext-0.13.1.dist-info/*
    /usr/local/lib/python3.7/dist-packages/torchtext/*
Proceed (y/n)? y
  Successfully uninstalled torchtext-0.13.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.0 MB 8.9 MB/s 
     |████████████████████████████████| 776.8 MB 17 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.1+cu113 requires torch==1.12.1, but you have torch 1.7.1 which is incompatible.
torchaudio 0.12.1+cu113 requires torch==1.12.1, but you have tor

In [128]:
!pip install torchmetrics
from torchmetrics.classification import BinaryF1Score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [129]:
!pip install -q captum
from captum.attr import LayerIntegratedGradients, TokenReferenceBase, visualization

In [2]:
import pandas as pd
import numpy as np
import torch

from torchtext import datasets

from torchtext.data import Field, LabelField
from torchtext.data import BucketIterator

from torchtext.vocab import Vectors, GloVe

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
from tqdm.autonotebook import tqdm

from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings('ignore')

В этом задании мы будем использовать библиотеку torchtext. Она довольна проста в использовании и поможет нам сконцентрироваться на задаче, а не на написании Dataloader-а.

In [3]:
TEXT = Field(sequential=True, lower=True, include_lengths=True)  # Поле текста
LABEL = LabelField(dtype=torch.float)  # Поле метки

In [4]:
SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

Датасет на котором мы будем проводить эксперементы это комментарии к фильмам из сайта IMDB.

In [5]:
train, test = datasets.IMDB.splits(TEXT, LABEL)  # загрузим датасет
train, valid = train.split(random_state=random.seed(SEED))  # разобьем на части

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:07<00:00, 10.5MB/s]


In [6]:
TEXT.build_vocab(train)
LABEL.build_vocab(train)

In [7]:
print("Size of TEXT vocabulary:",len(TEXT.vocab))
 
print("Size of LABEL vocabulary:",len(LABEL.vocab))
 
print(TEXT.vocab.freqs.most_common(10))  
 
#print(TEXT.vocab.stoi)

Size of TEXT vocabulary: 201849
Size of LABEL vocabulary: 2
[('the', 225611), ('a', 112054), ('and', 110751), ('of', 100826), ('to', 93398), ('is', 72993), ('in', 63227), ('i', 48804), ('this', 48462), ('that', 46254)]


In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"

train_iter, valid_iter, test_iter = BucketIterator.splits(
    (train, valid, test), 
    batch_size = 64,
    sort_within_batch = True,
    device = device)

In [9]:
len(train.examples[4].text)

177

## LSTM


In [12]:
class LSTM(nn.Module):
    
         # Определить все слои
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()          
        
                 #embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
                 #lstm слой
        self.rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
                           #batch_first=True)
        
                 # Полностью связанный слой
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
                 # Функция активации
        self.act = nn.Sigmoid()
        
    def forward(self, text, text_lengths):
        
        #text = [batch size,sent_length]
        embedded = self.embedding(text)
        #embedded = [batch size, sent_len, emb dim]
      
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.to('cpu'))#,batch_first=True)
        
#packed_sequences = nn.utils.rnn.pack_padded_sequence(padded_tensor, valid_frames.to('cpu'), batch_first=True, enforce_sorted=True)

        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
        
                 # Подключить конечное прямое и обратное скрытое состояние
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)
                
        #hidden = [batch size, hid dim * num directions]
        dense_outputs=self.fc(hidden)
 
                 # Активация
        outputs=self.act(dense_outputs)
        
        return outputs

In [13]:
vocab_size = len(TEXT.vocab)
embedding_dim = 200
hidden_dim = 256
output_dim = 1
n_layers = 2
bidirectional = True
dropout = 0.2
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
patience=3

In [14]:
model = LSTM(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    hidden_dim=hidden_dim,
    output_dim=output_dim,
    n_layers=n_layers,
    bidirectional=bidirectional,
    dropout=dropout,
    pad_idx=PAD_IDX
)

In [15]:
 # Если cuda доступна
model = model.to(device)

In [16]:
 # Определить оптимизатор и потерю
optimizer = optim.Adam(model.parameters())
criterion = nn.BCELoss()

max_epochs = 5

In [18]:
 # Определить показатели
def binary_accuracy(preds, y):
    # Округлить до ближайшего целого числа
    rounded_preds = torch.round(preds)
    
    correct = (rounded_preds == y).float() 
    acc = correct.sum() / len(correct)
    return acc

In [19]:
def train_lstm(model, iterator, optimizer, criterion):
  
  for epoch in range(1, max_epochs + 1):
      outputs = []
      targets = []
      train_loss = 0.0
      epoch_acc = 0
      model.train()
      pbar = tqdm(enumerate(iterator), total=len(iterator), leave=False)
      pbar.set_description(f"Epoch {epoch}")
      for it, batch in pbar: 
          
          optimizer.zero_grad()   
          
          text, text_lengths = batch.text   
          
                  # Преобразовать в одномерный тензор
          predictions = model(text, text_lengths).squeeze()
          
                  # Рассчитать значение функции потерь
          loss = criterion(predictions, batch.label)        
          
                  # Вычислить точность 
          acc = binary_accuracy(predictions, batch.label)  

                  
                  # Обратные потери при распространении и вычисление градиента
          loss.backward()       
          
                  # Обновить вес
          optimizer.step()      
          
                  # Потери и точность
          train_loss += loss.item()  
          epoch_acc += acc.item()    
          outputs.append(torch.round(predictions).cpu().int().numpy())
          targets.append(batch.label.cpu().int().numpy())

                      
      train_loss /= len(iterator)
      train_f1 = f1_score(np.concatenate(outputs), np.concatenate(targets), average = "micro")        
        
      print('Epoch: {}, Training Loss: {}'.format(epoch, train_loss))
      print('Epoch: {}, Training Acc: {}'.format(epoch, epoch_acc / len(train_iter), ))
      print('Epoch: {}, Training F_1: {}'.format(epoch, train_f1))      


In [20]:
def valid_lstm(model, iterator, criterion):
  
  min_loss = np.inf
  cur_patience = 0

  for epoch in range(1, max_epochs + 1):
      val_loss = 0.0
      val_epoch_acc = 0
      outputs_v = []
      targets_v = []
      model.eval()
      pbar = tqdm(enumerate(iterator), total=len(iterator), leave=False)
      pbar.set_description(f"Epoch {epoch}")
      
      for it, batch in pbar:
          text, text_lengths = batch.text
            
                        # Преобразовать в одномерный тензор
          predictions = model(text, text_lengths).squeeze()
          
                        # Расчет потерь и точности
          loss = criterion(predictions, batch.label)
          acc = binary_accuracy(predictions, batch.label)
                    
                        # Отслеживание потерь и точности
          val_loss += loss.item()
          val_epoch_acc += acc.item()
          outputs_v.append(torch.round(predictions).cpu().int().numpy())
          targets_v.append(batch.label.cpu().int().numpy())
        
    
      val_loss /= len(iterator)
      if val_loss < min_loss:
          min_loss = val_loss
          best_model = model.state_dict()
      else:
          cur_patience += 1
          if cur_patience == patience: 
              cur_patience = 0
              break

      val_f1 = f1_score(np.concatenate(outputs_v), np.concatenate(targets_v), average = "micro")
      print('Epoch: {}, Validation Loss: {}'.format(epoch, val_loss))
      print('Epoch: {}, Validation Acc: {}'.format(epoch, val_epoch_acc / len(valid_iter)))
      print('Epoch: {}, Validation F_1: {}'.format(epoch, val_f1))
    
  model.load_state_dict(best_model)

  

In [21]:
def test_lstm(model, iterator, criterion):

  val_loss = 0.0
  val_epoch_acc = 0
  outputs_v = []
  targets_v = []
  batch_count = 0
  model.eval()
  pbar = tqdm(enumerate(iterator), total=len(iterator), leave=False)
  
  with torch.no_grad():
    for it, batch in pbar:
        batch_count += 1
        text, text_lengths = batch.text
          
                      # Преобразовать в одномерный тензор
        predictions = model(text, text_lengths).squeeze()
        
                      # Расчет потерь и точности
        loss = criterion(predictions, batch.label)
        #print(loss)
        acc = binary_accuracy(predictions, batch.label)
        #print(acc)
                  
                      # Отслеживание потерь и точности
        val_loss += loss.item()
        #print(val_loss)
        val_epoch_acc += acc.item()
        #print(val_epoch_acc)
        outputs_v.append(torch.round(predictions).cpu().int().numpy())
        targets_v.append(batch.label.cpu().int().numpy())
              
      
  val_loss /= batch_count
  
  val_f1 = f1_score(np.concatenate(outputs_v), np.concatenate(targets_v), average = "micro")

  print('Test Loss: {}'.format(val_loss))
  print('Test Acc: {}'.format(val_epoch_acc / batch_count))
  print('Test F_1: {}'.format( val_f1))
      


  


In [22]:
print(train_lstm(model, train_iter, optimizer, criterion))


  0%|          | 0/274 [00:00<?, ?it/s]

Epoch: 1, Training Loss: 0.6387105333979113
Epoch: 1, Training Acc: 0.6314520335545505
Epoch: 1, Training F_1: 0.6314285714285715


  0%|          | 0/274 [00:00<?, ?it/s]

Epoch: 2, Training Loss: 0.49107473088006903
Epoch: 2, Training Acc: 0.7621220021787352
Epoch: 2, Training F_1: 0.7625142857142857


  0%|          | 0/274 [00:00<?, ?it/s]

Epoch: 3, Training Loss: 0.32630037347765734
Epoch: 3, Training Acc: 0.8601570647128307
Epoch: 3, Training F_1: 0.8599428571428571


  0%|          | 0/274 [00:00<?, ?it/s]

Epoch: 4, Training Loss: 0.15982975950804504
Epoch: 4, Training Acc: 0.9406689912298299
Epoch: 4, Training F_1: 0.9409142857142857


  0%|          | 0/274 [00:00<?, ?it/s]

Epoch: 5, Training Loss: 0.06761239147167245
Epoch: 5, Training Acc: 0.9777600364963503
Epoch: 5, Training F_1: 0.9777142857142858
None


In [23]:
print(valid_lstm(model, valid_iter, criterion))

  0%|          | 0/118 [00:00<?, ?it/s]

Epoch: 1, Validation Loss: 0.6783207147303274
Epoch: 1, Validation Acc: 0.7969632770045328
Epoch: 1, Validation F_1: 0.7978666666666666


  0%|          | 0/118 [00:00<?, ?it/s]

Epoch: 2, Validation Loss: 0.6783207147303274
Epoch: 2, Validation Acc: 0.7969632770045328
Epoch: 2, Validation F_1: 0.7978666666666666


  0%|          | 0/118 [00:00<?, ?it/s]

Epoch: 3, Validation Loss: 0.6783207116995827
Epoch: 3, Validation Acc: 0.7969632770045328
Epoch: 3, Validation F_1: 0.7978666666666666


  0%|          | 0/118 [00:00<?, ?it/s]

Epoch: 4, Validation Loss: 0.678320712709831
Epoch: 4, Validation Acc: 0.7969632770045328
Epoch: 4, Validation F_1: 0.7978666666666666


  0%|          | 0/118 [00:00<?, ?it/s]

None


In [24]:
print(test_lstm(model, test_iter, criterion))

  0%|          | 0/391 [00:00<?, ?it/s]

Test Loss: 0.7228367815122885
Test Acc: 0.7889865729814906
Test F_1: 0.78912
None


Посчитайте f1-score вашего классификатора на тестовом датасете.

**Ответ**: у модели LSTM на тесте получилась F1=0.79688

## CNN

![](https://www.researchgate.net/publication/333752473/figure/fig1/AS:769346934673412@1560438011375/Standard-CNN-on-text-classification.png)

Для классификации текстов также часто используют сверточные нейронные сети. Идея в том, что как правило сентимент содержат словосочетания из двух-трех слов, например "очень хороший фильм" или "невероятная скука". Проходясь сверткой по этим словам мы получим какой-то большой скор и выхватим его с помощью MaxPool. Далее идет обычная полносвязная сетка. Важный момент: свертки применяются не последовательно, а параллельно. Давайте попробуем!

In [106]:
TEXT = Field(sequential=True, lower=True, batch_first=True)  # batch_first тк мы используем conv  
LABEL = LabelField(batch_first=True, dtype=torch.float)

train, tst = datasets.IMDB.splits(TEXT, LABEL)
trn, vld = train.split(random_state=random.seed(SEED))

TEXT.build_vocab(trn)
LABEL.build_vocab(trn)

device = "cuda" if torch.cuda.is_available() else "cpu"

In [107]:
print(len(train))
print(len(vld))
print(len(tst))

25000
7500
25000


In [108]:
TEXT.vocab.itos[:20]

['<unk>',
 '<pad>',
 'the',
 'a',
 'and',
 'of',
 'to',
 'is',
 'in',
 'i',
 'this',
 'that',
 'it',
 '/><br',
 'was',
 'as',
 'for',
 'with',
 'but',
 'on']

In [109]:
train_iter, val_iter, test_iter = BucketIterator.splits(
        (trn, vld, tst),
        batch_sizes=(128, 256, 256),
        sort=False,
        sort_key= lambda x: len(x.src),
        sort_within_batch=False,
        device=device,
        repeat=False,
)

Вы можете использовать Conv2d с `in_channels=1, kernel_size=(kernel_sizes[0], emb_dim))` или Conv1d c `in_channels=emb_dim, kernel_size=kernel_size[0]`. Но хорошенько подумайте над shape в обоих случаях.

In [110]:
class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, dropout_proba):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)        
        self.conv_0 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(filter_sizes[0], embedding_dim))
        self.conv_1 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(filter_sizes[1], embedding_dim))
        self.conv_2 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(filter_sizes[2], embedding_dim))
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        self.dropout = nn.Dropout(dropout_proba)
        
    def forward(self, x):
        #x = [sent len, batch size]
        #x = x.permute(1, 0)
                
        #x = [batch size, sent len]
        embedded = self.embedding(x)
                
        #embedded = [batch size, sent len, emb dim]
        embedded = embedded.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim]
        conved_0 = F.relu(self.conv_0(embedded).squeeze(3))
        conved_1 = F.relu(self.conv_1(embedded).squeeze(3))
        conved_2 = F.relu(self.conv_2(embedded).squeeze(3))
            
        #conv_n = [batch size, n_filters, sent len - filter_sizes[n]]
        pooled_0 = F.max_pool1d(conved_0, conved_0.shape[2]).squeeze(2)
        pooled_1 = F.max_pool1d(conved_1, conved_1.shape[2]).squeeze(2)
        pooled_2 = F.max_pool1d(conved_2, conved_2.shape[2]).squeeze(2)
        
        #pooled_n = [batch size, n_filters]
        cat = self.dropout(torch.cat((pooled_0, pooled_1, pooled_2), dim=1))

        #cat = [batch size, n_filters * len(filter_sizes)]
        return self.fc(cat)

In [111]:
kernel_sizes = [3, 4, 5]
vocab_size = len(TEXT.vocab)
out_channels=64
dropout = 0.5
embed_dim = 300

model = CNN(vocab_size=vocab_size, embedding_dim=embed_dim, n_filters=out_channels,
            filter_sizes=kernel_sizes, dropout_proba=dropout, output_dim = 1)

In [112]:
model.to(device)

CNN(
  (embedding): Embedding(201849, 300)
  (conv_0): Conv2d(1, 64, kernel_size=(3, 300), stride=(1, 1))
  (conv_1): Conv2d(1, 64, kernel_size=(4, 300), stride=(1, 1))
  (conv_2): Conv2d(1, 64, kernel_size=(5, 300), stride=(1, 1))
  (fc): Linear(in_features=192, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [113]:
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()

In [114]:
max_epochs = 5

Обучите!

In [115]:
batch.label

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 1.,
        0., 1., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0., 1., 1., 0., 0., 0.,
        1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1.,
        1., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 0.,
        1., 0., 1., 1.], device='cuda:0')

In [116]:
batch.fields

dict_keys(['text', 'label'])

In [117]:
def binary_accuracy(preds, y):
    rounded_preds = torch.round(F.sigmoid(preds))
    correct = (rounded_preds == y).float()
    acc = correct.sum() / len(correct)
    return acc

In [118]:

min_loss = np.inf
cur_patience = 0
metric = BinaryF1Score().to(device)

for epoch in range(1, max_epochs + 1):
    train_loss = 0.0
    epoch_acc = 0.0
    train_f1_T = 0.0
    
    model.train()
    pbar = tqdm(enumerate(train_iter), total=len(train_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")
    for it, batch in pbar: 

        optimizer.zero_grad()   
        
        predictions = model(batch.text.cuda()).squeeze(1)
        loss = criterion(predictions.float(), batch.label.float().cuda())
        acc = binary_accuracy(predictions.float(), batch.label.float().cuda())
        
        loss.backward()       
        optimizer.step()      
        
        train_loss += loss.item()  
        epoch_acc += acc.item()    
        
        #print(metric(predictions, batch.label).cpu().numpy())
        train_f1_T += metric(predictions, batch.label).cpu().numpy()
        
    train_f1_T /= len(train_iter)
    train_loss /= len(train_iter)
    epoch_acc /= len(train_iter)
    
    val_loss = 0.0
    val_epoch_acc = 0.0
    val_f1_T = 0.0
    
    model.eval()
    pbar = tqdm(enumerate(val_iter), total=len(val_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")
    for it, batch in pbar:
                    
        predictions = model(batch.text.cuda()).squeeze(1)
        loss = criterion(predictions.float(), batch.label.float().cuda())
        acc = binary_accuracy(predictions.float(), batch.label.float().cuda())    
                           
        val_loss += loss.item()
        val_epoch_acc += acc.item()
        val_f1_T += metric(predictions, batch.label).cpu().numpy()


    val_f1_T /= len(val_iter)
    val_loss /= len(val_iter)
    val_epoch_acc /= len(val_iter)

    if val_loss < min_loss:
        min_loss = val_loss
        best_model = model.state_dict()
    else:
        cur_patience += 1
        if cur_patience == patience:
            cur_patience = 0
            break
    
    print('Epoch: {}, Training Loss: {}, Validation Loss: {}'.format(epoch, train_loss, val_loss))
    print('Epoch: {}, Training Acc: {}, Validation Acc: {}'.format(epoch, epoch_acc, val_epoch_acc))
    print('Epoch: {}, Training F1_T: {}, Validation F1_T: {}'.format(epoch, train_f1_T, val_f1_T))

model.load_state_dict(best_model)

  0%|          | 0/137 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch: 1, Training Loss: 0.6443013381348909, Validation Loss: 0.49195378124713895
Epoch: 1, Training Acc: 0.6354579895082182, Validation Acc: 0.7758497814337413
Epoch: 1, Training F1_T: 0.63071297337539, Validation F1_T: 0.7644917706648509


  0%|          | 0/137 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

...

Epoch: 4, Training Loss: 0.37618613612912866, Validation Loss: 0.36407814025878904
Epoch: 4, Training Acc: 0.8304481715181448, Validation Acc: 0.8402960518995921
Epoch: 4, Training F1_T: 0.8293016409351878, Validation F1_T: 0.842702841758728


  0%|          | 0/137 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch: 5, Training Loss: 0.3214976045772107, Validation Loss: 0.3433180759350459
Epoch: 5, Training Acc: 0.8662453389515842, Validation Acc: 0.8479303737481435
Epoch: 5, Training F1_T: 0.86547657869158, Validation F1_T: 0.855472731590271


<All keys matched successfully>

In [119]:
def test_cnn(model, iterator, criterion):

  val_loss = 0.0
  val_epoch_acc = 0
  val_f1_T = 0
  pred_cat = torch.empty(256) 
  pred_lab = torch.ones(256) 
  metric = BinaryF1Score().to(device)
  
  
  model.eval()
  pbar = tqdm(enumerate(iterator), total=len(iterator), leave=False)
  
  with torch.no_grad():
    for it, batch in pbar:
        
        predictions = model(batch.text.cuda()).squeeze(1)
        loss = criterion(predictions.float(), batch.label.float().cuda())
        acc = binary_accuracy(predictions.float(), batch.label.float().cuda())    
                           
        val_loss += loss.item()
        val_epoch_acc += acc.item()
        
        pred_cat = torch.cat((pred_cat.cuda(), predictions), 0)
        pred_lab = torch.cat((pred_lab.cuda(), batch.label), 0)
                     
  val_f1_T = metric(pred_cat, pred_lab).cpu().numpy()
  val_loss /= len(iterator)
  val_epoch_acc /= len(iterator)

  print('Test Loss: {}'.format(val_loss))
  print('Test Acc: {}'.format(val_epoch_acc))
  print('Test F_1: {}'.format(val_f1_T))
      

In [120]:
print(test_cnn(model, test_iter, criterion))

  0%|          | 0/98 [00:00<?, ?it/s]

Test Loss: 0.3397411640201296
Test Acc: 0.8530733725246118
Test F_1: 0.8521021604537964
None


Посчитайте f1-score вашего классификатора.

**Ответ**: Видим F1 = 0.85

## Интерпретируемость

Посмотрим, куда смотрит наша модель. Достаточно запустить код ниже.

In [138]:
PAD_IND = TEXT.vocab.stoi['pad']

token_reference = TokenReferenceBase(reference_token_idx=PAD_IND)
lig = LayerIntegratedGradients(model, model.embedding)

In [123]:
def forward_with_softmax(inp):
    logits = model(inp)
    return torch.softmax(logits, 0)[0][1]

def forward_with_sigmoid(input):
    return torch.sigmoid(model(input))


# accumalate couple samples in this array for visualization purposes
vis_data_records_ig = []

def interpret_sentence(model, sentence, min_len = 7, label = 0):
    model.eval()
    text = [tok for tok in TEXT.tokenize(sentence)]
    if len(text) < min_len:
        text += ['pad'] * (min_len - len(text))
    indexed = [TEXT.vocab.stoi[t] for t in text]

    model.zero_grad()

    input_indices = torch.tensor(indexed, device=device)
    input_indices = input_indices.unsqueeze(0)
    
    # input_indices dim: [sequence_length]
    seq_length = min_len

    # predict
    pred = forward_with_sigmoid(input_indices).item()
    pred_ind = round(pred)

    # generate reference indices for each sample
    reference_indices = token_reference.generate_reference(seq_length, device=device).unsqueeze(0)

    # compute attributions and approximation delta using layer integrated gradients
    attributions_ig, delta = lig.attribute(input_indices, reference_indices, \
                                           n_steps=5000, return_convergence_delta=True)

    print('pred: ', LABEL.vocab.itos[pred_ind], '(', '%.2f'%pred, ')', ', delta: ', abs(delta))

    add_attributions_to_visualizer(attributions_ig, text, pred, pred_ind, label, delta, vis_data_records_ig)
    
def add_attributions_to_visualizer(attributions, text, pred, pred_ind, label, delta, vis_data_records):
    attributions = attributions.sum(dim=2).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    attributions = attributions.cpu().detach().numpy()

    # storing couple samples in an array for visualization purposes
    vis_data_records.append(visualization.VisualizationDataRecord(
                            attributions,
                            pred,
                            LABEL.vocab.itos[pred_ind],
                            LABEL.vocab.itos[label],
                            LABEL.vocab.itos[1],
                            attributions.sum(),       
                            text,
                            delta))

In [124]:
interpret_sentence(model, 'It was a fantastic performance !', label=1)
interpret_sentence(model, 'Best film ever', label=1)
interpret_sentence(model, 'Such a great show!', label=1)
interpret_sentence(model, 'It was a horrible movie', label=0)
interpret_sentence(model, 'I\'ve never watched something as bad', label=0)
interpret_sentence(model, 'It is a disgusting movie!', label=0)

pred:  pos ( 0.87 ) , delta:  tensor([0.0001], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.05 ) , delta:  tensor([1.2269e-05], device='cuda:0', dtype=torch.float64)
pred:  pos ( 0.52 ) , delta:  tensor([5.8020e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.01 ) , delta:  tensor([4.8254e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.17 ) , delta:  tensor([1.1583e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.19 ) , delta:  tensor([5.3051e-05], device='cuda:0', dtype=torch.float64)


In [125]:
interpret_sentence(model, 'I will go crying after that!', label=0)
interpret_sentence(model, 'I like heroes, but i hate him.', label=0)
interpret_sentence(model, 'My mum does not hate the film.', label=1)
interpret_sentence(model, 'It was pretty and so good.', label=1)

pred:  neg ( 0.27 ) , delta:  tensor([9.9061e-06], device='cuda:0', dtype=torch.float64)
pred:  pos ( 0.57 ) , delta:  tensor([7.0600e-06], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.05 ) , delta:  tensor([2.6622e-06], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.34 ) , delta:  tensor([8.5532e-05], device='cuda:0', dtype=torch.float64)


Попробуйте добавить свои примеры!

In [126]:
print('Visualize attributions based on Integrated Gradients')
visualization.visualize_text(vis_data_records_ig)

Visualize attributions based on Integrated Gradients


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
pos,pos (0.87),pos,1.45,It was a fantastic performance ! pad
pos,neg (0.05),pos,1.33,Best film ever pad pad pad pad
pos,pos (0.52),pos,1.20,Such a great show! pad pad pad
neg,neg (0.01),pos,-0.26,It was a horrible movie pad pad
neg,neg (0.17),pos,0.73,I've never watched something as bad pad
neg,neg (0.19),pos,1.14,It is a disgusting movie! pad pad
neg,neg (0.27),pos,2.22,I will go crying after that! pad
neg,pos (0.57),pos,1.83,"I like heroes, but i hate him."
pos,neg (0.05),pos,0.51,My mum does not hate the film.
pos,neg (0.34),pos,1.17,It was pretty and so good. pad


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
pos,pos (0.87),pos,1.45,It was a fantastic performance ! pad
pos,neg (0.05),pos,1.33,Best film ever pad pad pad pad
pos,pos (0.52),pos,1.20,Such a great show! pad pad pad
neg,neg (0.01),pos,-0.26,It was a horrible movie pad pad
neg,neg (0.17),pos,0.73,I've never watched something as bad pad
neg,neg (0.19),pos,1.14,It is a disgusting movie! pad pad
neg,neg (0.27),pos,2.22,I will go crying after that! pad
neg,pos (0.57),pos,1.83,"I like heroes, but i hate him."
pos,neg (0.05),pos,0.51,My mum does not hate the film.
pos,neg (0.34),pos,1.17,It was pretty and so good. pad


## Эмбеддинги слов

Вы ведь не забыли, как мы можем применить знания о word2vec и GloVe. Давайте попробуем!

In [130]:
TEXT.build_vocab(trn, vectors="glove.42B.300d")# YOUR CODE GOES HERE
# подсказка: один из импортов пока не использовался, быть может он нужен в строке выше :)
LABEL.build_vocab(trn)

word_embeddings = TEXT.vocab.vectors

kernel_sizes = [3, 4, 5]
vocab_size = len(TEXT.vocab)
dropout = 0.5
dim = 300

In [131]:
train, tst = datasets.IMDB.splits(TEXT, LABEL)
trn, vld = train.split(random_state=random.seed(SEED))

device = "cuda" if torch.cuda.is_available() else "cpu"

train_iter, val_iter, test_iter = BucketIterator.splits(
        (trn, vld, tst),
        batch_sizes=(128, 256, 256),
        sort=False,
        sort_key= lambda x: len(x.src),
        sort_within_batch=False,
        device=device,
        repeat=False,
)

In [132]:
model = CNN(vocab_size=vocab_size, embedding_dim=embed_dim, n_filters=64,
            filter_sizes=kernel_sizes, dropout_proba=dropout, output_dim = 1)

word_embeddings = TEXT.vocab.vectors

prev_shape = model.embedding.weight.shape

model.embedding.weight = nn.Parameter(word_embeddings)

assert prev_shape == model.embedding.weight.shape
model.to(device)

optimizer = torch.optim.Adam(model.parameters())

In [133]:
def test_cnn_em(model, iterator, criterion):

  val_loss = 0.0
  val_epoch_acc = 0
  val_f1_T = 0
  pred_cat = torch.empty(256) 
  pred_lab = torch.ones(256) 
  metric = BinaryF1Score().to(device)
  
  model.eval()
  pbar = tqdm(enumerate(iterator), total=len(iterator), leave=False)
  
  with torch.no_grad():
    for it, batch in pbar:
        
        predictions = model(batch.text.cuda()).squeeze(1)
        loss = criterion(predictions.float(), batch.label.float().cuda())
        acc = binary_accuracy(predictions.float(), batch.label.float().cuda())    
                           
        val_loss += loss.item()
        val_epoch_acc += acc.item()
        
        pred_cat = torch.cat((pred_cat.cuda(), predictions), 0)
        pred_lab = torch.cat((pred_lab.cuda(), batch.label), 0)
                     
  val_f1_T = metric(pred_cat, pred_lab).cpu().numpy()
  val_loss /= len(iterator)
  val_epoch_acc /= len(iterator)

  print('Test Loss: {}'.format(val_loss))
  print('Test Acc: {}'.format(val_epoch_acc))
  print('Test F_1: {}'.format(val_f1_T))
      

Вы знаете, что делать.

In [134]:
min_loss = np.inf
cur_patience = 0
metric = BinaryF1Score().to(device)
patience = 3

for epoch in range(1, max_epochs + 1):
    train_loss = 0.0
    epoch_acc = 0.0
    train_f1_T = 0.0
    
    model.train()
    pbar = tqdm(enumerate(train_iter), total=len(train_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")
    for it, batch in pbar: 

        optimizer.zero_grad()   
        
        predictions = model(batch.text.cuda()).squeeze(1)
        loss = criterion(predictions.float(), batch.label.float().cuda())
        acc = binary_accuracy(predictions.float(), batch.label.float().cuda())
        
        loss.backward()       
        optimizer.step()      
        
        train_loss += loss.item()  
        epoch_acc += acc.item()    
        
        #print(metric(predictions, batch.label).cpu().numpy())
        train_f1_T += metric(predictions, batch.label).cpu().numpy()
        
    train_f1_T /= len(train_iter)
    train_loss /= len(train_iter)
    epoch_acc /= len(train_iter)
    
    val_loss = 0.0
    val_epoch_acc = 0.0
    val_f1_T = 0.0
    
    model.eval()
    pbar = tqdm(enumerate(val_iter), total=len(val_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")
    for it, batch in pbar:
                    
        predictions = model(batch.text.cuda()).squeeze(1)
        loss = criterion(predictions.float(), batch.label.float().cuda())
        acc = binary_accuracy(predictions.float(), batch.label.float().cuda())    
                           
        val_loss += loss.item()
        val_epoch_acc += acc.item()
        val_f1_T += metric(predictions, batch.label).cpu().numpy()


    val_f1_T /= len(val_iter)
    val_loss /= len(val_iter)
    val_epoch_acc /= len(val_iter)

    if val_loss < min_loss:
        min_loss = val_loss
        best_model = model.state_dict()
    else:
        cur_patience += 1
        if cur_patience == patience:
            cur_patience = 0
            break
    
    print('Epoch: {}, Training Loss: {}, Validation Loss: {}'.format(epoch, train_loss, val_loss))
    print('Epoch: {}, Training Acc: {}, Validation Acc: {}'.format(epoch, epoch_acc, val_epoch_acc))
    print('Epoch: {}, Training F1_T: {}, Validation F1_T: {}'.format(epoch, train_f1_T, val_f1_T))

model.load_state_dict(best_model)

  0%|          | 0/137 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch: 1, Training Loss: 0.49819619159628875, Validation Loss: 0.35597441891829174
Epoch: 1, Training Acc: 0.7487330410602319, Validation Acc: 0.8512815237045288
Epoch: 1, Training F1_T: 0.7391229073931701, Validation F1_T: 0.8481917679309845


  0%|          | 0/137 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

...

Epoch: 4, Training Loss: 0.08519215865509354, Validation Loss: 0.2850330258409182
Epoch: 4, Training Acc: 0.9757765392317389, Validation Acc: 0.8879797160625458
Epoch: 4, Training F1_T: 0.9756378944772873, Validation F1_T: 0.8883486151695251


  0%|          | 0/137 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch: 5, Training Loss: 0.035919507543971066, Validation Loss: 0.3000043367346128
Epoch: 5, Training Acc: 0.9928147810218978, Validation Acc: 0.8895422160625458
Epoch: 5, Training F1_T: 0.9927735972578509, Validation F1_T: 0.8915823578834534


<All keys matched successfully>

In [135]:
print(test_cnn(model, test_iter, criterion))

  0%|          | 0/98 [00:00<?, ?it/s]

Test Loss: 0.3083940890370583
Test Acc: 0.8808176176888602
Test F_1: 0.8744516968727112
None


Посчитайте f1-score вашего классификатора.

**Ответ**: получили 0.87

Проверим насколько все хорошо!

In [136]:
PAD_IND = TEXT.vocab.stoi['pad']

token_reference = TokenReferenceBase(reference_token_idx=PAD_IND)
lig = LayerIntegratedGradients(model, model.embedding)
vis_data_records_ig = []

interpret_sentence(model, 'It was a fantastic performance !', label=1)
interpret_sentence(model, 'Best film ever', label=1)
interpret_sentence(model, 'Such a great show!', label=1)
interpret_sentence(model, 'It was a horrible movie', label=0)
interpret_sentence(model, 'I\'ve never watched something as bad', label=0)
interpret_sentence(model, 'It is a disgusting movie!', label=0)

pred:  pos ( 0.90 ) , delta:  tensor([1.1070e-05], device='cuda:0', dtype=torch.float64)
pred:  pos ( 0.84 ) , delta:  tensor([4.2535e-05], device='cuda:0', dtype=torch.float64)
pred:  pos ( 1.00 ) , delta:  tensor([3.4471e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.02 ) , delta:  tensor([8.3970e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.43 ) , delta:  tensor([3.1266e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.07 ) , delta:  tensor([9.5623e-05], device='cuda:0', dtype=torch.float64)


In [137]:
print('Visualize attributions based on Integrated Gradients')
visualization.visualize_text(vis_data_records_ig)

Visualize attributions based on Integrated Gradients


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
pos,pos (0.90),pos,0.02,It was a fantastic performance ! pad
pos,pos (0.84),pos,-0.81,Best film ever pad pad pad pad
pos,pos (1.00),pos,1.42,Such a great show! pad pad pad
neg,neg (0.02),pos,-1.23,It was a horrible movie pad pad
neg,neg (0.43),pos,-2.12,I've never watched something as bad pad
neg,neg (0.07),pos,-0.99,It is a disgusting movie! pad pad


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
pos,pos (0.90),pos,0.02,It was a fantastic performance ! pad
pos,pos (0.84),pos,-0.81,Best film ever pad pad pad pad
pos,pos (1.00),pos,1.42,Such a great show! pad pad pad
neg,neg (0.02),pos,-1.23,It was a horrible movie pad pad
neg,neg (0.43),pos,-2.12,I've never watched something as bad pad
neg,neg (0.07),pos,-0.99,It is a disgusting movie! pad pad


## Вывод:## 
Видим, что наилучшие результаты (F1=0.8744) показыла сверточная нейронная сеть с предобученными эмбеддингами слов из GloVe.